# Reasoning Direction

This notebook aims to estimate the "reasoning" direction within the LLM activation space.
We're basing this approach on the methodology used to find the "refusal" direction, but with a key difference:

- **Refusal paper approach**: Used 1 LLM with 2 types of prompts (harmful vs harmless)
- **Our approach**: Use 2 models (original vs reasoning-tuned) with the same prompts (GSM8K math problems)

We'll collect activations from both models, calculate the difference (reasoning direction),
and then test if adding this direction to the reasoning/non-reasoning model enhances its reasoning capabilities.

In [1]:
# Setup and imports
!pip install transformers transformers_stream_generator tiktoken transformer_lens einops jaxtyping colorama scikit-learn datasets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 70.0 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 KB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 KB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 KB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 KB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 110.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 KB 60.1 M

In [10]:
!pip uninstall numpy -y

Found existing installation: numpy 1.21.5
Not uninstalling numpy at /usr/lib/python3/dist-packages, outside environment /usr
Can't uninstall 'numpy'. No files were found to uninstall.


In [1]:
import torch
import functools
import einops
import requests
import pandas as pd
import io
import textwrap
import gc
import transformers
import numpy as np

from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch import Tensor
from typing import List, Callable, Dict, Tuple, Optional, Union
from transformer_lens import HookedTransformer, utils
from transformer_lens.hook_points import HookPoint
from transformers import AutoTokenizer, AutoModelForCausalLM
from jaxtyping import Float, Int
from colorama import Fore
from huggingface_hub import snapshot_download

# We turn off automatic differentiation to save GPU memory
torch.set_grad_enabled(False)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-26 17:09:39.016428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740589779.087523   10796 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740589779.108412   10796 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load models

We'll load both the original model and the reasoning-tuned model using HookedTransformer.
If using a HuggingFace model, we can download it directly. If using a local model, make sure it's 
in the correct directory structure.

In [2]:
# Define model paths - adjust these based on your models
MODEL_PATH_ORIGINAL = "meta-llama/Llama-3.1-8B-Instruct"  # Non-reasoning model
MODEL_PATH_REASONING = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"  # Reasoning model

### Load both models

In [4]:
# Login to HuggingFace
from huggingface_hub import login

# Replace with your HF token from https://huggingface.co/settings/tokens
login(token="") 

In [5]:
# Load the original (non-reasoning) model
model_original = HookedTransformer.from_pretrained_no_processing(
    MODEL_PATH_ORIGINAL,
    dtype=torch.bfloat16,
    default_padding_side='left'
)
model_original.tokenizer.padding_side = 'left'
model_original.tokenizer.pad_token = model_original.tokenizer.eos_token

print(f"Loaded non-reasoning model {MODEL_PATH_ORIGINAL}")

Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.49it/s]


Loaded pretrained model meta-llama/Llama-3.1-8B-Instruct into HookedTransformer
Loaded non-reasoning model meta-llama/Llama-3.1-8B-Instruct


In [31]:
# # free cuda memory
# torch.cuda.empty_cache()
# gc.collect()


127

#### Optional: Download the reasoning model if needed
This step can be skipped if you already have the models locally

In [6]:
# Uncomment this to download the reasoning model
model_path = snapshot_download(
    repo_id=MODEL_PATH_REASONING,
    local_dir=MODEL_PATH_ORIGINAL,
    local_dir_use_symlinks=False
)

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 11 files:   9%|▉         | 1/11 [00:00<00:01,  6.20it/s]

Fetching 11 files: 100%|██████████| 11/11 [13:53<00:00, 75.77s/it]


In [7]:
# Load the reasoning model
model_reasoning = HookedTransformer.from_pretrained_no_processing(
    MODEL_PATH_ORIGINAL,
    local_files_only=True,  # Set to True if using local models
    dtype=torch.bfloat16,
    default_padding_side='left'
)
model_reasoning.tokenizer.padding_side = 'left'
model_reasoning.tokenizer.pad_token = model_reasoning.tokenizer.eos_token

print(f"Loaded reasoning model {MODEL_PATH_REASONING}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.18it/s]


Loaded pretrained model meta-llama/Llama-3.1-8B-Instruct into HookedTransformer
Loaded reasoning model deepseek-ai/DeepSeek-R1-Distill-Llama-8B


In [8]:
# Test both models with a simple math problem
test_prompt = "What is 5*125?"

# Test original model
print("Original model response:")
output_original = model_original.generate(
    test_prompt, 
    temperature=0.0,
    max_new_tokens=100  # Increase max tokens to generate longer response
)
print(output_original)

print("\nReasoning model response:") 
output_reasoning = model_reasoning.generate(
    test_prompt,
    temperature=0.0,
    max_new_tokens=100  # Increase max tokens to generate longer response
)
print(output_reasoning)


Original model response:


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:05<00:00, 17.35it/s]


What is 5*125? 5*125 = 625
What is 5*1250? 5*1250 = 6250
What is 5*12500? 5*12500 = 62500
What is 5*125000? 5*125000 = 625000
What is 5*1250000? 5*1250000 = 6250000
What is 5*12500000? 5*12500000 = 

Reasoning model response:


100%|██████████| 100/100 [00:05<00:00, 19.29it/s]

What is 5*125? Let me think. Okay, 5 times 100 is 500, and 5 times 25 is 125. So if I add those together, 500 plus 125 is 625. Hmm, that seems right. Let me double-check by breaking it down another way. 125 times 5 is the same as 125 plus 125 plus 125 plus 125 plus 125. So, 125 plus 125 is 250, plus another 125 is 375


## Set up chat templates and data processing functions

We need to define chat templates for both models and create functions to process our data.
Different models may have different chat templates, so we adjust accordingly.

In [9]:
# Define chat templates for both models
# Adjust these templates based on your specific models
ORIGINAL_CHAT_TEMPLATE = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>
{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""  # Llama-3 template

REASONING_CHAT_TEMPLATE = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>
{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""  # DeepSeek template

In [10]:
# Define utility functions for processing data and collecting activations

def tokenize_instructions(
    tokenizer: AutoTokenizer,
    instructions: List[str],
    chat_template: str
) -> Int[Tensor, 'batch_size seq_len']:
    """Tokenize instructions using the specified chat template."""
    prompts = [chat_template.format(instruction=instruction) for instruction in instructions]
    return tokenizer(prompts, padding=True, truncation=False, return_tensors="pt").input_ids

def collect_activations(
    model: HookedTransformer,
    tokenized_inputs: Int[Tensor, 'batch_size seq_len'],
    batch_size: int = 8
) -> Dict[str, Tensor]:
    """Collect activations from a model for the given inputs."""
    activations = {}
    
    for i in tqdm(range(0, len(tokenized_inputs), batch_size)):
        batch = tokenized_inputs[i:i+batch_size]
        
        # Run the model and cache activations
        logits, cache = model.run_with_cache(
            batch, 
            names_filter=lambda hook_name: 'resid' in hook_name, 
            device='cpu'  # Use CPU to avoid OOM errors; switch to 'cuda' if you have enough VRAM
        )
        
        # First batch, initialize the dictionary
        if not activations:
            activations = {key: [cache[key]] for key in cache}
        else:
            # Append to existing cache
            for key in cache:
                activations[key].append(cache[key])
                
        # Clear memory
        del logits, cache
        gc.collect()
        torch.cuda.empty_cache()
    
    # Concatenate all batches
    activations = {k: torch.cat(v) for k, v in activations.items()}
    return activations

## Load and prepare GSM8K dataset

We'll use the GSM8K dataset which contains math problems that require reasoning to solve.
We'll append "please provide your answer first, then your reasoning" to each problem.

In [11]:
# Load GSM8K dataset
gsm8k = load_dataset("gsm8k", "main")
print(f"Loaded GSM8K dataset with {len(gsm8k['train'])} training examples and {len(gsm8k['test'])} test examples")

# Look at a sample problem
print("\nSample problem:")
print(gsm8k["train"][0]["question"])
print("\nSample answer:")
print(gsm8k["train"][0]["answer"])

Generating test split: 100%|██████████| 1319/1319 [00:00<00:00, 295134.01 examples/s]

Loaded GSM8K dataset with 7473 training examples and 1319 test examples

Sample problem:
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

Sample answer:
Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72


In [32]:
# Define functions to prepare prompts
def prepare_prompts(problems: List[str]) -> List[str]:
    """Add the reasoning instruction to each problem."""
    return [f"{problem}\n\nPlease provide your answer first, then your reasoning." for problem in problems]

# Prepare prompts for training and testing
train_problems = [item["question"] for item in gsm8k["train"]]
test_problems = [item["question"] for item in gsm8k["test"]]

# Limit the number of problems to reduce computation time
N_PROBLEMS = 100  # Adjust based on available compute
train_problems = train_problems[:N_PROBLEMS]
test_problems = test_problems[:min(20, len(test_problems))]  # Smaller test set

train_prompts = prepare_prompts(train_problems)
test_prompts = prepare_prompts(test_problems)


print(f"Prepared {len(train_prompts)} training prompts and {len(test_prompts)} test prompts")

Prepared 100 training prompts and 20 test prompts


## Collect activations from both models

Now we'll run the same prompts through both models and collect their activations.
This is the key step where we gather the data needed to compute the reasoning direction.

In [13]:
# Tokenize the math problems for both models
tokenized_prompts_original = tokenize_instructions(
    model_original.tokenizer, 
    train_prompts, 
    ORIGINAL_CHAT_TEMPLATE
)

tokenized_prompts_reasoning = tokenize_instructions(
    model_reasoning.tokenizer, 
    train_prompts, 
    REASONING_CHAT_TEMPLATE
)

print(f"Tokenized prompts shape (original): {tokenized_prompts_original.shape}")
print(f"Tokenized prompts shape (reasoning): {tokenized_prompts_reasoning.shape}")

Tokenized prompts shape (original): torch.Size([100, 134])
Tokenized prompts shape (reasoning): torch.Size([100, 140])


In [ ]:
# Create a list of simple toy math problems
toy_math_problems = [
    "What is 2+2?",
    "What is 5*7?",
    "If x=3 and y=4, what is x+y?",
    "What is the square root of 16?",
    "What is 10 divided by 2?",
    "If I have 3 apples and get 5 more, how many do I have?",
    "What is 9 squared?",
    "What is 15-7?",
    "What is the product of 4 and 6?",
    "If a rectangle has length 5 and width 3, what is its area?"
]

toy_prompts = prepare_prompts(toy_math_problems)


### Collect activations (this may take a while)
We'll run both models on the same inputs and collect their activations.

In [14]:
print("Collecting activations from the original model...")
original_activations = collect_activations(model_original, tokenized_prompts_original)
print("Done collecting activations from the original model")

print("Collecting activations from the reasoning model...")
reasoning_activations = collect_activations(model_reasoning, tokenized_prompts_reasoning)
print("Done collecting activations from the reasoning model")

# Optional: Save activations to disk to avoid recomputing
torch.save(original_activations, 'original_activations.pth')
torch.save(reasoning_activations, 'reasoning_activations.pth')

100%|██████████| 13/13 [00:13<00:00,  1.00s/it]


Done collecting activations from the original model


100%|██████████| 13/13 [00:08<00:00,  1.48it/s]


Done collecting activations from the reasoning model


## Calculate the reasoning direction

Now we'll calculate the reasoning direction by taking the difference between
activations from the reasoning model and the original model.

In [15]:
def get_act_idx(cache_dict, act_name, layer):
    """Helper function to get activations from a specific layer."""
    key = (act_name, layer,)
    return cache_dict[utils.get_act_name(*key)]

# The activation layers to analyze
activation_layers = ['resid_pre', 'resid_mid', 'resid_post']

# Calculate reasoning directions
reasoning_directions = {k: [] for k in activation_layers}

for layer_num in tqdm(range(1, model_original.cfg.n_layers)):
    pos = -1  # Focus on the last token position
    
    for layer in activation_layers:
        # Get mean activations for each model
        original_mean_act = get_act_idx(original_activations, layer, layer_num)[:, pos, :].mean(dim=0)
        reasoning_mean_act = get_act_idx(reasoning_activations, layer, layer_num)[:, pos, :].mean(dim=0)
        
        # Calculate the difference and normalize to get the direction
        reasoning_dir = reasoning_mean_act - original_mean_act
        reasoning_dir = reasoning_dir / reasoning_dir.norm()
        
        reasoning_directions[layer].append(reasoning_dir)

# Save the reasoning directions
torch.save(reasoning_directions, 'reasoning_dirs.pth')
print("Reasoning directions calculated and saved")

100%|██████████| 31/31 [00:00<00:00, 119.11it/s]

Reasoning directions calculated and saved


## Score and rank reasoning directions

Now we'll sort the reasoning directions by their magnitude to identify
which layers might have the strongest reasoning signal.

In [16]:
# Get all calculated potential reasoning dirs, sort them in descending order
# based on their mean() magnitude
activation_layers = ['resid_pre']  # We can start with just this layer as it's often sufficient

# Flatten and score all directions
activation_scored = sorted(
    [reasoning_directions[layer][l-1] for l in range(1, model_original.cfg.n_layers) for layer in activation_layers], 
    key=lambda x: abs(x.mean()), 
    reverse=True
)

print(f"Ranked {len(activation_scored)} potential reasoning directions")

Ranked 31 potential reasoning directions


## Test the reasoning direction

Now we'll define a hook to add the reasoning direction to the model's activations
during inference and test if it enhances the model's reasoning capabilities.

In [24]:
def reasoning_enhancement_hook(
    activation: Float[Tensor, "... d_act"],
    hook: HookPoint,
    direction: Float[Tensor, "d_act"],
    strength: float = 1.0
):
    """Hook to add the reasoning direction to activations."""
    # Ensure direction is on same device as activation
    if activation.device != direction.device:
        direction = direction.to(activation.device)
    
    # Project the activation onto the reasoning direction and add it back
    proj = einops.einsum(activation, direction, '... d_act, d_act -> ...')
    return activation + (strength * direction.unsqueeze(0).unsqueeze(0))

def generate_with_hooks(
    model: HookedTransformer,
    toks: Int[Tensor, 'batch_size seq_len'],
    max_tokens_generated: int = 100,
    fwd_hooks = [],
    temperature: float = 0.0
) -> List[str]:
    """Generate text with specified hooks applied."""
    device = toks.device  # Get the device of input tokens
    
    all_toks = torch.zeros(
        (toks.shape[0], toks.shape[1] + max_tokens_generated), 
        dtype=torch.long, 
        device=device  # Ensure this tensor is on the same device
    )
    all_toks[:, :toks.shape[1]] = toks
    
    # Keep track of which sequences have finished generating
    finished = torch.zeros(toks.shape[0], dtype=torch.bool, device=device)  # Match device
    eos_token_id = model.tokenizer.eos_token_id
    
    for i in range(max_tokens_generated):
        with model.hooks(fwd_hooks=fwd_hooks):
            logits = model(all_toks[:, :-max_tokens_generated + i])
            if temperature == 0.0:
                next_tokens = logits[:, -1, :].argmax(dim=-1)  # greedy sampling
            else:
                probs = torch.nn.functional.softmax(logits[:, -1, :] / temperature, dim=-1)
                next_tokens = torch.multinomial(probs, num_samples=1).squeeze(-1)
            
            # Make sure next_tokens is on the same device
            next_tokens = next_tokens.to(device)
            
            # Only update unfinished sequences
            all_toks[:, -max_tokens_generated + i] = torch.where(
                finished, 
                all_toks[:, -max_tokens_generated + i], 
                next_tokens
            )
            
            # Mark sequences as finished if they generated an EOS token
            finished = finished | (next_tokens == eos_token_id)
            
            # Stop if all sequences are finished
            if finished.all():
                break
    
    # For each sequence, only return the generated tokens up to any EOS token
    results = []
    for b in range(all_toks.shape[0]):
        # Get generated tokens for this batch item
        gen_toks = all_toks[b, toks.shape[1]:]
        
        # Find the first EOS token position
        eos_pos = (gen_toks == eos_token_id).nonzero()
        
        # If EOS found, truncate at that position
        if len(eos_pos) > 0:
            gen_toks = gen_toks[:eos_pos[0].item()]
        
        # Decode to text
        results.append(model.tokenizer.decode(gen_toks, skip_special_tokens=True))
    
    return results

## Evaluate the reasoning enhancement

Now we'll test our reasoning direction on a few example problems and compare
the baseline model outputs with the reasoning-enhanced outputs.

In [19]:
# Select the top reasoning direction to test
top_reasoning_dir = activation_scored[0]
print("Selected top reasoning direction for testing")

# Create hooks to inject reasoning direction
strength = 0.1  # Adjust this value to control the magnitude of the effect
hook_fn = functools.partial(reasoning_enhancement_hook, direction=top_reasoning_dir, strength=strength)

# Create hooks for all layers (or you can target specific layers)
fwd_hooks = [
    (utils.get_act_name(act_name, l), hook_fn) 
    for l in range(model_original.cfg.n_layers) 
    for act_name in ['resid_pre', 'resid_mid', 'resid_post']
]

Selected top reasoning direction for testing


In [35]:
# Test on a few examples
N_TEST_EXAMPLES = 10
print(f"Testing on {N_TEST_EXAMPLES} examples from the test set")

for i in range(N_TEST_EXAMPLES):
    test_prompt = toy_math_problems[i]
    print(f"\n\n--- EXAMPLE {i+1} ---")
    print(f"PROBLEM:\n{test_prompt}")
    
    # Tokenize the test prompt
    test_tokens = tokenize_instructions(
        model_original.tokenizer, 
        [test_prompt], 
        ORIGINAL_CHAT_TEMPLATE
    )
    
    # Generate baseline response (without reasoning enhancement)
    baseline_response = generate_with_hooks(model_reasoning, test_tokens, max_tokens_generated=500)
    print("\nBASELINE RESPONSE:")
    print(baseline_response[0])
    
    # Generate reasoning-enhanced response
    enhanced_response = generate_with_hooks(model_reasoning, test_tokens, fwd_hooks=fwd_hooks, max_tokens_generated=500)
    print("\nREASONING-ENHANCED RESPONSE:")
    print(enhanced_response[0])

Testing on 10 examples from the test set


--- EXAMPLE 1 ---
PROBLEM:
What is 2+2?

BASELINE RESPONSE:
<think>
Okay, so I need to figure out what 2 plus 2 equals. Hmm, let me think about this. I remember from school that addition is a basic math operation, right? It's when you add two numbers together. So, 2 plus 2 should be... let me count on my fingers. One, two, three. Yeah, that makes sense. So, 2 plus 2 equals 4. I think that's right, but maybe I should double-check to make sure I'm not making a mistake. Sometimes I get confused with subtraction, but addition is straightforward. Yeah, I'm pretty sure it's 4.
</think>

2 + 2 equals 4.

REASONING-ENHANCED RESPONSE:
<think>
Okay, so I'm trying to figure out what "2+2" means. I've come across this phrase in some online discussions or maybe even in some gaming chats. I'm not entirely sure what it means, but I think it's something related to math or maybe even gaming. Let me try to break it down.

First, I know that "2" is the number tw

## Systematic Evaluation

To properly evaluate the effectiveness of our reasoning direction,
we'll test it on more examples and compare the quality of the responses.

In [36]:
def evaluate_reasoning(
    model: HookedTransformer,
    problems: List[str],
    chat_template: str,
    reasoning_hooks=None,
    max_tokens: int = 150,
    batch_size: int = 4
) -> List[Dict]:
    """Evaluate model performance with and without reasoning enhancement."""
    results = []
    
    for i in tqdm(range(0, len(problems), batch_size)):
        batch_problems = problems[i:i+batch_size]
        tokens = tokenize_instructions(model.tokenizer, batch_problems, chat_template)
        
        # Generate without reasoning enhancement
        baseline_responses = generate_with_hooks(model, tokens, max_tokens_generated=max_tokens)
        
        # Generate with reasoning enhancement if hooks provided
        if reasoning_hooks:
            enhanced_responses = generate_with_hooks(
                model, tokens, max_tokens_generated=max_tokens, fwd_hooks=reasoning_hooks
            )
        else:
            enhanced_responses = ["No enhancement applied"] * len(batch_problems)
        
        # Store results
        for j, (problem, baseline, enhanced) in enumerate(
            zip(batch_problems, baseline_responses, enhanced_responses)
        ):
            results.append({
                "problem": problem,
                "baseline": baseline,
                "enhanced": enhanced
            })
    
    return results

In [37]:
# Evaluate on a larger test set
evaluation_results = evaluate_reasoning(
    model_reasoning,
    test_prompts[:10],  # Use more examples for a better evaluation
    ORIGINAL_CHAT_TEMPLATE,
    reasoning_hooks=fwd_hooks
)

100%|██████████| 3/3 [01:51<00:00, 37.02s/it]


In [38]:
# Print and analyze evaluation results
for i, result in enumerate(evaluation_results):
    print(f"\n--- EVALUATION EXAMPLE {i+1} ---")
    print(f"PROBLEM:\n{result['problem']}")
    print(f"\nBASELINE SOLUTION:\n{result['baseline']}")
    print(f"\nREASONING-ENHANCED SOLUTION:\n{result['enhanced']}")
    print("-" * 80)


--- EVALUATION EXAMPLE 1 ---
PROBLEM:
Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

Please provide your answer first, then your reasoning.

BASELINE SOLUTION:

Okay, so I need to figure out how much money Janet makes every day from selling her duck eggs. Let me break this down step by step.

First, Janet's ducks lay 16 eggs per day. That's the total number of eggs she gets each day. Now, she does a few things with these eggs: she eats three for breakfast every morning, she bakes muffins for her friends using four eggs each day, and then she sells the remaining eggs at the farmers' market for $2 each. I need to find out how much she makes from selling those remaining eggs.

Alright, let's start by calculating the total eggs laid. She lays 16 eggs each da

## Future Directions

Some potential improvements and extensions to this work:

1. **Fine-tune the strength parameter**: Experiment with different values of the strength parameter
2. **Layer-specific intervention**: Apply the reasoning direction to specific layers only
3. **Quantitative evaluation**: Develop metrics to measure reasoning quality
4. **Orthogonalization**: Create a model with permanent reasoning enhancement by orthogonalizing the weights
5. **Multiple reasoning directions**: Identify different aspects of reasoning (deduction, induction, etc.)

This approach of comparing model activations to find meaningful directions in the activation space
could be extended to many other capabilities beyond reasoning.